In [71]:
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

essays = pd.read_csv('AllFeaturesAllSets.csv')
essays.dropna(axis=0, how='any', inplace=True)

X = essays.drop(['domain1_score','essay'], axis=1)
y = essays[['domain1_score','essay_set']]
y.iloc[0]['domain1_score']

8

In [72]:
def get_score(essay_set, score):
        if essay_set == 1:
            div = 12/3
        elif essay_set == 2:
            div = 5/3
        elif essay_set == 3:
            div = 3/3
        elif essay_set == 4:
            div = 3/3
        elif essay_set == 5:
            div = 4/3
        elif essay_set == 6:
            div = 4/3
        elif essay_set == 7:
            div = 25/3
        elif essay_set == 8:
            div = 50/3
        return score/float(div)

In [73]:
arr = []
for i in range(0,len(y)):
    score = get_score(y.iloc[i]['essay_set'],y.iloc[i]['domain1_score'])
    arr.append(score)

y = y.assign(score=arr)
y.head()

,domain1_score,essay_set,score
0,8,1,2.00
1,9,1,2.25
2,7,1,1.75
3,10,1,2.50
4,8,1,2.00


In [74]:

Xsets = [None] * 9
ysets = [None] * 9
for i in range(0,8):
    Xsets[i] = X.loc[X['essay_set'] == i+1]
    Xsets[i] = Xsets[i].drop(['essay_set'], axis=1)
    ysets[i] = y.loc[y['essay_set'] == i+1]
    ysets[i] = ysets[i].drop(['essay_set','domain1_score'], axis=1)

ysets[6].tail()

,score
12257,1.44
12258,1.92
12259,2.28
12260,2.64
12261,1.80


In [75]:
from sklearn.metrics import (confusion_matrix,
                             f1_score,
                             make_scorer,
                             SCORERS)
def kappa(y_true, y_pred, weights=None, allow_off_by_one=False):
    """
    Calculates the kappa inter-rater agreement between two the gold standard
    and the predicted ratings. Potential values range from -1 (representing
    complete disagreement) to 1 (representing complete agreement).  A kappa
    value of 0 is expected if all agreement is due to chance.

    In the course of calculating kappa, all items in ``y_true`` and ``y_pred`` will
    first be converted to floats and then rounded to integers.

    It is assumed that y_true and y_pred contain the complete range of possible
    ratings.

    This function contains a combination of code from yorchopolis's kappa-stats
    and Ben Hamner's Metrics projects on Github.

    Parameters
    ----------
    y_true : array-like of float
        The true/actual/gold labels for the data.
    y_pred : array-like of float
        The predicted/observed labels for the data.
    weights : str or np.array, optional
        Specifies the weight matrix for the calculation.
        Options are ::

            -  None = unweighted-kappa
            -  'quadratic' = quadratic-weighted kappa
            -  'linear' = linear-weighted kappa
            -  two-dimensional numpy array = a custom matrix of

        weights. Each weight corresponds to the
        :math:`w_{ij}` values in the wikipedia description
        of how to calculate weighted Cohen's kappa.
        Defaults to None.
    allow_off_by_one : bool, optional
        If true, ratings that are off by one are counted as
        equal, and all other differences are reduced by
        one. For example, 1 and 2 will be considered to be
        equal, whereas 1 and 3 will have a difference of 1
        for when building the weights matrix.
        Defaults to False.

    Returns
    -------
    k : float
        The kappa score, or weighted kappa score.

    Raises
    ------
    AssertionError
        If ``y_true`` != ``y_pred``.
    ValueError
        If labels cannot be converted to int.
    ValueError
        If invalid weight scheme.
    """

    # Ensure that the lists are both the same length
    assert(len(y_true) == len(y_pred))

    # This rather crazy looking typecast is intended to work as follows:
    # If an input is an int, the operations will have no effect.
    # If it is a float, it will be rounded and then converted to an int
    # because the ml_metrics package requires ints.
    # If it is a str like "1", then it will be converted to a (rounded) int.
    # If it is a str that can't be typecast, then the user is
    # given a hopefully useful error message.
    try:
        y_true = [int(np.round(float(y))) for y in y_true]
        y_pred = [int(np.round(float(y))) for y in y_pred]
    except ValueError:
        raise ValueError("For kappa, the labels should be integers or strings "
                         "that can be converted to ints (E.g., '4.0' or '3').")

    # Figure out normalized expected values
    min_rating = min(min(y_true), min(y_pred))
    max_rating = max(max(y_true), max(y_pred))

    # shift the values so that the lowest value is 0
    # (to support scales that include negative values)
    y_true = [y - min_rating for y in y_true]
    y_pred = [y - min_rating for y in y_pred]

    # Build the observed/confusion matrix
    num_ratings = max_rating - min_rating + 1
    observed = confusion_matrix(y_true, y_pred,
                                labels=list(range(num_ratings)))
    num_scored_items = float(len(y_true))

    # Build weight array if weren't passed one
    if isinstance(weights, str):
        wt_scheme = weights
        weights = None
    else:
        wt_scheme = ''
    if weights is None:
        weights = np.empty((num_ratings, num_ratings))
        for i in range(num_ratings):
            for j in range(num_ratings):
                diff = abs(i - j)
                if allow_off_by_one and diff:
                    diff -= 1
                if wt_scheme == 'linear':
                    weights[i, j] = diff
                elif wt_scheme == 'quadratic':
                    weights[i, j] = diff ** 2
                elif not wt_scheme:  # unweighted
                    weights[i, j] = bool(diff)
                else:
                    raise ValueError('Invalid weight scheme specified for '
                                     'kappa: {}'.format(wt_scheme))

    hist_true = np.bincount(y_true, minlength=num_ratings)
    hist_true = hist_true[: num_ratings] / num_scored_items
    hist_pred = np.bincount(y_pred, minlength=num_ratings)
    hist_pred = hist_pred[: num_ratings] / num_scored_items
    expected = np.outer(hist_true, hist_pred)

    # Normalize observed array
    observed = observed / num_scored_items

    # If all weights are zero, that means no disagreements matter.
    k = 1.0
    if np.count_nonzero(weights):
        k -= (sum(sum(weights * observed)) / sum(sum(weights * expected)))

    return k

In [76]:
# !pip install skll.metrics
# import skll
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import (confusion_matrix,
                             f1_score,
                             make_scorer,
                             SCORERS)



for i in range(0,8):
    clf = svm.SVR(kernel = 'rbf')
    X_train, X_test, y_train, y_test = train_test_split(Xsets[i], ysets[i], test_size=0.2, random_state=1)
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    #print(y_pred)
    y_test=y_test.to_numpy(dtype=float, copy=False)
    cappa_score = 0
    
    for j in range(0,len(y_test)):
        #print(y_test[j],y_pred[j])
        cappa_score+= kappa([y_test[j]],[y_pred[j]],weights='quadratic')
#         print(kappa([y_test[j]],[y_pred[j]],weights='quadratic'))
        
    
    avg_cappa_score = cappa_score/len(y_test)
    print('Kappa score for Set ')
    print(i+1)
    print( avg_cappa_score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Kappa score for Set 
1
0.8907563025210085


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Kappa score for Set 
2
0.8972222222222223


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Kappa score for Set 
6
0.725


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Kappa score for Set 
7
0.7006369426751592
Kappa score for Set 
8
0.8


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [5]:

from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score

clf = svm.SVR(kernel = 'rbf')

X_train, X_test, y_train, y_test = train_test_split(Xsets[0], ysets[0], test_size=0.2, random_state=1)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print(y_pred[:5])
cappa_score = 0
#     for j in range(0,len(y_test)):
#         cappa_score+= cohen_kappa_score([y_test[j]],[y_pred[j]],weights='quadratic')
#     avg_cappa_score = cappa_score/len(y_test)
#     print('Kappa score for Set '+i+1+' is '+avg_cappa_score+'\n')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


[7.92853288 7.95074822 9.3371221  8.1455362  9.94057708]
